## Project 2

Необходимо реализовать модулярную арифметику, с помощью класса Mod(value, modulus) и реализовать арифметические операции между объектами.

Более подробно ознакомиться с условиями можно в 63 уроке курса Python 3: Deep Dive (Part 4 - OOP)

In [85]:
import types
import numbers
from functools import total_ordering

In [129]:
@total_ordering
class Mod():
    'Modular arithmetic'
    def __init__(self, value, modulus):
        if not self._validate_value(modulus):
            raise ValueError('Modulus must be Integer')
        if modulus <= 0:
            raise ValueError('Modulus must be positive')
        self._modulus = modulus
        
        if not self._validate_value(value):
            raise ValueError('Value must be Integer')
        self._value = value % modulus
        self._real_value = value
    
    #read-only properties
    @property
    def value(self):
        return self._value
    
    @property
    def modulus(self):
        return self._modulus
    
    #representation methods
    def __repr__(self):
        return f'Mod({self.value}, {self.modulus})'
    
    def __int__(self):
        return self.value
    
    #arithmetic operations
    def __add__(self, other):
        other_value = self._get_value(other)
        return Mod(self.value + other_value, self.modulus)
    
    def __radd__(self, other):
        return self + other
    
    def __iadd__(self, other):
        res = self + other
        self._value = res._value
        self._real_value = res._real_value
        return self
    
    def __sub__(self, other):
        other_value = self._get_value(other)
        return Mod(self.value - other_value, self.modulus)
    
    def __rsub__(self, other):
        return -(self - other)
    
    def __isub__(self, other):
        res = self - other
        self._value = res._value
        self._real_value = res._real_value
        return self
    
    def __mul__(self, other):
        other_value = self._get_value(other)
        return Mod(self.value * other_value, self.modulus)
    
    def __rmul__(self, other):
        return self * other
    
    def __imul__(self, other):
        res = self * other
        self._value = res._value
        self._real_value = res._real_value
        return self
    
    def __pow__(self, other):
        if self._validate_value(other):
            return self ** Mod(other, self.modulus)
        if not self._validate_value(other, Mod) or (other.modulus != self.modulus):
            return NotImplemented
        return Mod(self._real_value ** other._real_value, self.modulus)
    
    def __rpow__(self, other):
        if self._validate_value(other):
            return Mod(other, self.modulus) ** self
        else:
            return NotImplemented
    
    def __ipow__(self, other):
        res = self ** other
        self._value = res._value
        self._real_value = res._real_value
        return self
    
    def __neg__(self):
        return Mod(-self.value, self.modulus)
    
    #order methods
    def __eq__(self, other):
        other_value = self._get_value(other)
        return self.value == other_value
    
    def __lt__(self, other):
        other_value = self._get_value(other)
        return self.value < other_value
    
    #validate static method
    @staticmethod
    def _validate_value(value, type_value=numbers.Integral):
        return isinstance(value, type_value)
    
    def _get_value(self, other):
        if self._validate_value(other):
            return other % self.modulus
        if self._validate_value(other, Mod) and (self.modulus == other.modulus):
            return other.value
        raise TypeError('Not implemented')

## Tests

In [132]:
import unittest

In [133]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [146]:
class TestMod(unittest.TestCase):
    def test_create_mod(self):
        with self.assertRaises(ValueError):
            Mod(None, 10)
        
        with self.assertRaises(ValueError):
            Mod(5, 0)
            
        with self.assertRaises(ValueError):
            Mod(5, -20)
            
        with self.assertRaises(ValueError):
            Mod('5', 20)
            
        with self.assertRaises(ValueError):
            Mod(2.5, 10)
        
        with self.assertRaises(ValueError):
            Mod(2, 10.1)
    
    def test_arithmetic_operation_equal(self):
        a = Mod(3, 10)
        
        self.assertEqual(a + 5, Mod(8, 10))
        self.assertEqual(a + Mod(5, 10), Mod(8, 10))
        self.assertEqual(5 + a, Mod(8, 10))
        self.assertEqual(a - 2, Mod(1, 10))
        self.assertEqual(a - Mod(2, 10), Mod(1, 10))
        self.assertEqual(5 - a, Mod(2, 10))
        self.assertEqual(a * 2, Mod(6, 10))
        self.assertEqual(2 * a, Mod(6, 10))
        self.assertEqual(a * Mod(2, 10), Mod(6, 10))
        self.assertEqual(a ** 2, Mod(9, 10))
        self.assertEqual(2 ** a, Mod(8, 10))
        self.assertEqual(a ** Mod(2, 10), Mod(9, 10))
        self.assertEqual(a ** Mod(3, 10), Mod(7, 10))
    
    def test_order_methods_equal(self):
        a = Mod(7, 15)
        
        self.assertTrue(a == a)
        self.assertTrue(a == Mod(7, 15))
        self.assertTrue(a == (Mod(3, 15) + Mod(4, 15)))
        self.assertTrue(a == (Mod(10, 15) - Mod(3, 15)))
        self.assertFalse(a == (Mod(10, 15) - Mod(5, 15)))
        self.assertFalse(a == (a + 1))
        self.assertFalse(a < Mod(7, 15))
        self.assertTrue(a < Mod(10, 15))
        self.assertTrue(a < (2 * a))
        
        with self.assertRaises(TypeError):
            a < 10.5
        
        with self.assertRaises(TypeError):
            a < Mod(7, 10)

In [147]:
run_tests(TestMod)

test_arithmetic_operation_equal (__main__.TestMod) ... ok
test_create_mod (__main__.TestMod) ... ok
test_order_methods_equal (__main__.TestMod) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
